# Problem 3 - Ray Tune for Hyperparameter Optimization

## 3.1

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from ray import tune
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from time import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

import ray
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.tune.search.bayesopt import BayesOptSearch

2023-12-05 11:52:12.760326: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 11:52:18.569590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 11:52:18.569626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 11:52:18.585610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 11:52:18.627292: I tensorflow/core/platform/cpu_feature_guar

In [10]:
def train_mnist(config):
    # Load MNIST data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # Add a channels dimension
    x_train = x_train[..., tf.newaxis].astype("float32")
    x_test = x_test[..., tf.newaxis].astype("float32")

    # Define the model
    model = Sequential([
        Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(config["dropout"]),
        Dense(10, activation="softmax")
    ])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Train the model
    model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=config["batch_size"],
        epochs=10,
        callbacks=[TuneReportCallback({"accuracy": "accuracy"})]
    )

# Define the search space
search_space = {
    "conv_filters": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size": tune.choice([64, 128, 256]),
    "dropout": tune.uniform(0, 1)
}

# Initialize Ray
ray.init(ignore_reinit_error=True)

2023-12-05 12:14:55,573	INFO worker.py:1507 -- Calling ray.init() again after it has already been called.


Python version:,3.11.5
Ray version:,2.8.1


In [11]:
def train_mnist(config):
    num_classes = 10
    epochs = 12

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 28, 28, 1) / 255.0, x_test.reshape(-1, 28, 28, 1) / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        metrics=["accuracy"])

    for epoch in range(epochs):
        model.fit(
            x_train,
            y_train,
            batch_size=config["batch_size"],
            epochs=1,
            verbose=0,
            validation_data=(x_test, y_test))

        # Evaluate the model
        i, accuracy = model.evaluate(x_test, y_test, verbose=0)
        session.report({"mean_accuracy": accuracy})

### Grid 

In [12]:
# Grid Search
start_time = time()

grid_analysis = tune.run(
    train_mnist,
    name="exp",
    metric="mean_accuracy",
    mode="max",
    stop={"mean_accuracy": 0.99},
    resources_per_trial={"gpu": 1},
    config={
        "conv_filters": tune.grid_search([64, 128, 256]),
        "lr": tune.grid_search([0.001, 0.01, 0.1]),
        "batch_size": tune.grid_search([64, 128, 256]),
        "dropout": tune.grid_search([0.0, 0.25, 0.5, 0.75, 0.9]),
    }
)

end_time = time()
grid_time = end_time - start_time

2023-12-05 12:14:58,081	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-05 12:14:58,096	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=3

Trial name,mean_accuracy
train_mnist_d0fea_00000,0.9875
train_mnist_d0fea_00001,0.9879
train_mnist_d0fea_00002,0.9875
train_mnist_d0fea_00003,0.988
train_mnist_d0fea_00004,0.9871
train_mnist_d0fea_00005,0.9862
train_mnist_d0fea_00006,0.9871
train_mnist_d0fea_00007,0.9862
train_mnist_d0fea_00008,0.9871
train_mnist_d0fea_00009,0.9883


(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=3

In [13]:
best_trial = grid_analysis.get_best_trial("mean_accuracy", "max", "last")
best_config = best_trial.config

print("Best trial config: {}".format(best_config))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["mean_accuracy"]))
print("Time taken for Grid Search: {} seconds".format(grid_time))
# Naming error; takes too long to run but fixed the error in the code

Best trial config: {'conv_filters': 64, 'lr': 0.001, 'batch_size': 256, 'dropout': 0.25}
Best trial final validation accuracy: 0.9894000291824341
Time taken for Grid Search: 7111.168742179871 seconds


### Bayesian

In [23]:
def train_mnist_bayes(config):
    # Convert continuous parameters back to categorical
    #conv_filters_map = {0: 64, 1: 128, 2: 256}
    batch_size_map = {0: 64, 1: 128, 2: 256}

    config["conv_filters"] = int(round(config["conv_filters"]))
    config["batch_size"] = batch_size_map[int(round(config["batch_size"]))]

    # Load MNIST data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 28, 28, 1) / 255.0, x_test.reshape(-1, 28, 28, 1) / 255.0

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=config["conv_filters"], kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(config["dropout"]),
        tf.keras.layers.Dense(10, activation="softmax")  # Assuming 10 classes for MNIST
    ])

    # Compile the model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"]),
        metrics=["accuracy"]
    )

    # Train the model
    model.fit(
        x_train,
        y_train,
        batch_size=config["batch_size"],
        epochs=12,  # or any other number of epochs you wish to use
        verbose=1,
        validation_data=(x_test, y_test)
    )

    # Evaluate the model
    i, accuracy = model.evaluate(x_test, y_test, verbose=0)
    session.report({"mean_accuracy": accuracy})

In [31]:
bayes_search_space = {
    "conv_filters": tune.uniform(64,256),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size": tune.uniform(0, 2),  # 0 for 64, 1 for 128, 2 for 256
    "dropout": tune.uniform(0, 1)
}

# Initialize Bayesian optimization search algorithm
bayesopt_search = BayesOptSearch()

start_time = time()

bayes_analysis = tune.run(
    train_mnist_bayes,
    name="exp_bayes",
    metric="mean_accuracy",
    mode="max",
    stop={"mean_accuracy": 0.99},
    resources_per_trial={"gpu": 1},
    config=search_space,
    search_alg=bayesopt_search,
    num_samples=10
)

end_time = time()
bayes_time = end_time - start_time

2023-12-05 09:44:19,935	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-05 09:44:19,952	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2023-12-05 09:44:19,954	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=2

(train_mnist pid=2617307) Epoch 1/12


(train_mnist pid=2617307) 2023-12-05 09:44:27.622406: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
(train_mnist pid=2617307) 2023-12-05 09:44:38.600964: I external/local_xla/xla/service/service.cc:168] XLA service 0x14d3902f69f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
(train_mnist pid=2617307) 2023-12-05 09:44:38.601007: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 8000, Compute Capability 7.5
(train_mnist pid=2617307) 2023-12-05 09:44:38.610090: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
(train_mnist pid=2617307) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(train_mnist pid=2617307) I0000 00:00:1701787478.845065 2617902 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most

469/469 [==============================] - 21s 8ms/step - loss: 3.1074 - accuracy: 0.1059 - val_loss: 2.3034 - val_accuracy: 0.1135
(train_mnist pid=2617307) Epoch 2/12
469/469 [==============================] - 3s 6ms/step - loss: 2.3054 - accuracy: 0.1076 - val_loss: 2.3043 - val_accuracy: 0.1135
(train_mnist pid=2617307) Epoch 3/12
469/469 [==============================] - 3s 6ms/step - loss: 2.3054 - accuracy: 0.1071 - val_loss: 2.3050 - val_accuracy: 0.1135
(train_mnist pid=2617307) Epoch 4/12
469/469 [==============================] - 3s 6ms/step - loss: 2.3052 - accuracy: 0.1059 - val_loss: 2.3076 - val_accuracy: 0.1135
(train_mnist pid=2617307) Epoch 5/12
469/469 [==============================] - 3s 6ms/step - loss: 2.3056 - accuracy: 0.1051 - val_loss: 2.3058 - val_accuracy: 0.1032
(train_mnist pid=2617307) Epoch 6/12
469/469 [==============================] - 3s 6ms/step - loss: 2.3046 - accuracy: 0.1074 - val_loss: 2.3058 - val_accuracy: 0.1009
(train_mnist pid=2617307) Ep

...

(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=2

(train_mnist pid=2645324) Epoch 1/12


(train_mnist pid=2645324) 2023-12-05 09:50:57.172051: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
(train_mnist pid=2645324) 2023-12-05 09:50:58.147996: I external/local_xla/xla/service/service.cc:168] XLA service 0x1499f830bc80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
(train_mnist pid=2645324) 2023-12-05 09:50:58.148045: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 8000, Compute Capability 7.5
(train_mnist pid=2645324) 2023-12-05 09:50:58.157088: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
(train_mnist pid=2645324) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(train_mnist pid=2645324) I0000 00:00:1701787858.281410 2645459 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most

469/469 [==============================] - 5s 5ms/step - loss: 2.4091 - accuracy: 0.1085 - val_loss: 2.3056 - val_accuracy: 0.1028
(train_mnist pid=2645324) Epoch 2/12
469/469 [==============================] - 2s 4ms/step - loss: 2.3041 - accuracy: 0.1071 - val_loss: 2.3045 - val_accuracy: 0.0980
(train_mnist pid=2645324) Epoch 3/12
469/469 [==============================] - 2s 5ms/step - loss: 2.3041 - accuracy: 0.1085 - val_loss: 2.3059 - val_accuracy: 0.0982
(train_mnist pid=2645324) Epoch 4/12
469/469 [==============================] - 2s 5ms/step - loss: 2.3038 - accuracy: 0.1070 - val_loss: 2.3046 - val_accuracy: 0.1135
(train_mnist pid=2645324) Epoch 5/12
469/469 [==============================] - 2s 5ms/step - loss: 2.3043 - accuracy: 0.1068 - val_loss: 2.3027 - val_accuracy: 0.1135
(train_mnist pid=2645324) Epoch 6/12
469/469 [==============================] - 2s 5ms/step - loss: 2.3041 - accuracy: 0.1064 - val_loss: 2.3040 - val_accuracy: 0.1028
(train_mnist pid=2645324) Epo

2023-12-05 09:51:25,501	INFO tune.py:1047 -- Total run time: 425.57 seconds (425.53 seconds for the tuning loop).


In [33]:
best_trial = bayes_analysis.get_best_trial("mean_accuracy", "max", "last")
best_config = best_trial.config

print("Best trial config: {}".format(best_config))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["mean_accuracy"]))
print("Time taken for Bayesian Search: {} seconds".format(bayes_time))

Best trial config: {'conv_filters': 164.75323487338966, 'lr': 0.029831684879606152, 'batch_size': 0.6084844859190754, 'dropout': 0.43194501864211576}
Best trial final validation accuracy: 0.9726999998092651
Time taken for Bayesian Search: 425.58768105506897 seconds


### Hyperband 

In [4]:
hyperband_search_space = {
    "conv_filters": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(0.001, 0.1),
    "batch_size": tune.choice([64, 128, 256]),
    "dropout": tune.uniform(0, 1)
}

In [5]:
from ray.tune.schedulers import HyperBandScheduler

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Start time
start_time = time()

# Define the HyperBandScheduler with metric and mode
hyperband_scheduler = HyperBandScheduler(metric="mean_accuracy", mode="max")

# Run Hyperband optimization without metric and mode in tune.run()
hyperband_analysis = tune.run(
    train_mnist,
    name="hyperband_exp",
    stop={"mean_accuracy": 0.99, "training_iteration": 10},
    resources_per_trial={"gpu": 1},
    config=hyperband_search_space,
    num_samples=20,  # Number of different hyperparameter combinations to try
    scheduler=hyperband_scheduler
)

# End time
end_time = time()
hyperband_time = end_time - start_time

2023-12-05 11:54:08,350	INFO worker.py:1507 -- Calling ray.init() again after it has already been called.
2023-12-05 11:54:08,354	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-12-05 11:54:08,367	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2023-12-05 11:54:08,368	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=3

Trial name,mean_accuracy
train_mnist_e8192_00000,0.9865
train_mnist_e8192_00001,0.9887
train_mnist_e8192_00002,0.098
train_mnist_e8192_00003,0.9839
train_mnist_e8192_00004,0.9527
train_mnist_e8192_00005,0.1009
train_mnist_e8192_00006,0.9888
train_mnist_e8192_00007,0.9447
train_mnist_e8192_00008,0.984
train_mnist_e8192_00009,0.9528


(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 3 of /home/gha2009/.local/lib/python3.11/site-packages/googleapis_common_protos-1.61.0-py3.9-nspkg.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "<frozen site>", line 186, in addpackage
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "<frozen importlib._bootstrap>", line 570, in module_from_spec
(raylet)   AttributeError: 'NoneType' object has no attribute 'loader'
(raylet) 
(raylet) Remainder of file ignored
(pid=3

In [6]:
best_trial = hyperband_analysis.get_best_trial("mean_accuracy", "max", "last")
best_config = best_trial.config

print("Best trial config: {}".format(best_config))
print("Best trial final validation accuracy: {}".format(
    best_trial.last_result["mean_accuracy"]))
print("Time taken for Hyperband Search: {} seconds".format(hyperband_time))

Best trial config: {'conv_filters': 64, 'lr': 0.0021042765420631734, 'batch_size': 128, 'dropout': 0.3736771928672201}
Best trial final validation accuracy: 0.9887999892234802
Time taken for Hyperband Search: 1210.3595538139343 seconds


## 3.2

In [8]:
def display_results(analysis, technique_name, time_taken):
    best_trial = analysis.get_best_trial("mean_accuracy", "max", "last")
    best_config = best_trial.config
    best_accuracy = best_trial.last_result["mean_accuracy"]

    print(f"{technique_name} Results:")
    print(f"Time Taken: {time_taken:.2f} seconds")
    print(f"Best Hyperparameters: {best_config}")
    print(f"Best Validation Accuracy: {best_accuracy:.4f}")
    print("-" * 30)

In [14]:
# Display results for Grid Search
display_results(grid_analysis, "Grid Search", grid_time)

Grid Search Results:
Time Taken: 7111.17 seconds
Best Hyperparameters: {'conv_filters': 64, 'lr': 0.001, 'batch_size': 256, 'dropout': 0.25}
Best Validation Accuracy: 0.9894
------------------------------


In [36]:
# Display results for Bayesian Search
display_results(bayes_analysis, "Bayesian Search", bayes_time)

Bayesian Search Results:
Time Taken: 425.59 seconds
Best Hyperparameters: {'conv_filters': 164.75323487338966, 'lr': 0.029831684879606152, 'batch_size': 0.6084844859190754, 'dropout': 0.43194501864211576}
Best Validation Accuracy: 0.9727
------------------------------


In [9]:
# Display results for Hyperband
display_results(hyperband_analysis, "Hyperband", hyperband_time)

Hyperband Results:
Time Taken: 1210.36 seconds
Best Hyperparameters: {'conv_filters': 64, 'lr': 0.0021042765420631734, 'batch_size': 128, 'dropout': 0.3736771928672201}
Best Validation Accuracy: 0.9888
------------------------------


## 3.3

**Answer:**

Analyzing your results, several key observations can be made regarding the time taken for hyperparameter optimization and the performance of the best model identified by each search method:

Grid Search took the longest time (7111.17 seconds) among the three methods. This is expected as grid search exhaustively explores the defined hyperparameter space, which can be time-consuming, especially for large search spaces. The best model achieved a high validation accuracy of 0.9894. This indicates that despite its time-consuming nature, grid search can be effective in finding well-performing hyperparameters.

Bayesian Search method was significantly faster (425.59 seconds), demonstrating Bayesian search's efficiency in navigating the hyperparameter space by building a probabilistic model and using it to select the most promising hyperparameters. The best model had a lower validation accuracy (0.9727) compared to grid search. This could be due to the Bayesian search exploring more diverse regions of the hyperparameter space, which might lead to finding good but not necessarily the best parameters within the limited number of trials.

Hyperband was faster than grid search but slower than Bayesian search (1210.36 seconds). Hyperband, being a bandit-based approach, efficiently allocates resources to promising configurations and quickly discards poor-performing ones, which can lead to time savings. The best model identified by Hyperband achieved a validation accuracy of 0.9888, which is very close to Grid Search yet better than Bayesian Search. This suggests a good balance between exploration and exploitation in Hyperband's approach.

In general, there is a clear trade-off between the time taken for hyperparameter optimization and the performance of the best model. Grid search, while time-consuming, found the most accurate model. Bayesian search, despite being much faster, compromised slightly on accuracy. Bayesian search and Hyperband offer more efficient hyperparameter tuning compared to grid search, though they may require careful tuning of their own parameters to achieve the best results. Overall, the choice of method depends on the specific requirements and constraints of the project goal. If computational resources and time are limited, Bayesian search or Hyperband might be more suitable. If the highest possible accuracy is paramount and resources are ample, grid search might be the better choice. Regardless, all accuracy scores are high and very close to each other, so the specific case in this scenario depends more. 